In [1]:
import pandas as pd
import numpy as np
from translate import Translator
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soubhik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from translate import Translator
translator = Translator(from_lang="bn",to_lang="en")

In [3]:
def LCstring(X, Y):
    m=len(X)
    n=len(Y)
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)]
    result = 0
    for i in range(m + 1):
        for j in range(n + 1):
            if (i == 0 or j == 0):
                LCSuff[i][j] = 0
            elif (X[i-1] == Y[j-1]):
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j])
            else:
                LCSuff[i][j] = 0
    return result

In [4]:
def LCsequence(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)

    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]

    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
    return L[m][n]


In [5]:
#fetch suffix
suffix=pd.read_excel('suffix_set.xlsx')
class1_suffix=suffix[suffix['weight']==1]
class2_suffix=suffix[suffix['weight']==2]
class3_suffix=suffix[suffix['weight']==3]
# hybrid_suffix=suffix[suffix['weight'].isin([3,4])]

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [273]:
dictnary=pd.read_excel('root_words.xlsx')

In [274]:
def checking(test_word):
    global prediction_list
    if dictnary['word'].isin([test_word]).any():
        prediction_list.append(test_word)
    else:
        return(True)

In [275]:
def checking_with_translation(left_context,test_word,right_context):
    root=[0.0,'']
    translation_word=translator.translate(test_word).lower()
    actual_window=left_context+" "+test_word+" "+right_context
    translation_window=translator.translate(actual_window).lower()
#     if(LCstring(translation_word,translation_window)/len(translation_word)>0.7):
#         for predicted_word in prediction_list:
#             translation_testword=translator.translate(predicted_word).lower()
#             print(translation_testword,predicted_word)
#             print(translation_word,test_word)
#             accurecy=LCstring(translation_word,translation_testword)/len(translation_testword)
#             print(LCstring(translation_word,translation_testword))
#             print(accurecy)
#             if(accurecy>=root[0]):
#                 root[0]=accurecy
#                 root[1]=predicted_word
#     else:
    for predicted_word in prediction_list:
        test_window=left_context+" "+predicted_word+" "+right_context
        translation_testwindow=translator.translate(test_window).lower()
        accurecy=LCsequence(translation_window,translation_testwindow)/len(translation_testwindow)
#       print(translation_testwindow,accurecy)
        if accurecy>root[0]:
            root[0]=accurecy
            root[1]=predicted_word
        elif accurecy==root[0]:
            if(LCsequence(test_word,root[1])<=LCsequence(test_word,predicted_word)):
                root[0]=accurecy
                root[1]=predicted_word
                
    return(root[1])
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword)))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/len(translation_testword))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/abs(len(translation_testword)-len(translation_window)))
#     print(abs(len(translation_testword)-len(translation_window))/LCsequence(translation_window,translation_testword))

In [276]:
def predict_root(suffix,test_word):
    predicted_root=[test_word+suffix for suffix in suffix.suffix]
    for i in predicted_root:
        if checking(i):
            continue
    return predicted_root

In [277]:
def remove_suffix(target_suffix,word):
    final_suffix=''
    #find actual suffix 
    #find longest suffix bettwen target_suffix that belongs at the end of given word 
    for i in target_suffix:
        count=0
        for j in range(-1,(-(len(i)+1)),-1):
            if (word[j]==i[j]):
                count+=1
            else:
                break
        if(count==len(i)):
            final_suffix=i
            break
        #remove suffix
    if final_suffix:
        SL_word=word.rstrip(final_suffix)    
        return (SL_word)
    else:
        return (word)

In [278]:
def leematizer(context_word):
    global prediction_list
    container=[]
    word=context_word
    container.append(word)
    print("input ",word)
    checking(word)
    #checking for direct root
    if (class1_suffix['suffix'].isin([word[-1]]).any()):
        #if there is any suffix of class 1 remove it
        SL_word=remove_suffix(word[-1],word)
        # SL_word denoted suffix_less_word
        checking(SL_word)
        predict_root(class1_suffix,SL_word)
    index=[i in word for i  in class3_suffix.suffix]
    # select class 3 suffix
    if any(index):
        # sort target suffix list in desc order of length
        target_suffix=class3_suffix[index].suffix.tolist()
        target_suffix.sort(key=len,reverse=True)
        SL_word=remove_suffix(target_suffix,word)
        #remove suffix
        #SL_word denoted suffix_less_word
        checking(SL_word)
        word=SL_word
        #if it is a root word declear rootword else check is there any other suffix present
        container.append(word)
    predict_root(class2_suffix,word) # select only class 1 suffix suffix that is in the word
    index=[]
    if len(word)>1 and (class1_suffix['suffix'].isin([word[-1]]).any()): #remove class 1 suffix suffix from end
        SL_word=remove_suffix(word[-1],word)
        checking(SL_word)
        #if Sl_word is root word declear it or checck it with other suffix one by one
        word=SL_word
    predicted_subword=predict_root(class1_suffix,word)
    for i in predicted_subword:
        predict_root(class2_suffix,i)
    prediction_list=[*set(prediction_list)]   #remove duplicate prediction 
    print(prediction_list)
    if(len(prediction_list)==1): #if predicttion list contain only one prediction declear as root
        print(context_word,'--->',prediction_list[0])
    elif(len(prediction_list)>1): #if more than one check using meaning
        root=checking_with_translation(left_context,context_word,right_context)
        print(context_word,'--->',root)

In [285]:
words=[]
#sentence1="ছাত্রজীবন মানব জীবনের প্রস্তুতিকাল। ছাত্রজীবনের ভিত্তির উপর ধীরে ধীরে মানবজীবনের কর্ম কৃতিত্বের সৌধটি গড়ে ওঠে। এই ভিত্তি সুদৃঢ় হবে, ভবিষ্যতে জীবন ও তত বলিষ্ঠ এবং সমৃদ্ধ হবে। মানব জীবনের ভিত্তি সুদৃঢ় করতে হলে ছাত্রজীবনে যত্নবান হতে হবে।তাই ছাত্রজীবনে লেখাপড়াও খেলাধুলার সঙ্গে সঙ্গে যেমন শরীর ও চরিত্র গঠন করতে হবে, তেমনি জগৎ ও জীবনের বহু বিচিত্র কর্মধারার সঙ্গে পরিচয় সাধন করতে হবে। এই পরিচয় সাধনের সূত্রই ছাত্রজীবনে সমাজকল্যাণ অথবা সমাজসেবা।"
#sentence1="সভ্যতার প্রথম উন্মেষ কালে আগুন আবিষ্কারের পরই সমাজ গঠন বোধকরি মানব সভ্যতার অন্যতম অবদান। তারপর থেকে সভ্যতার অগ্রগতি ও ক্রমবিকাশের সঙ্গে সঙ্গে গোষ্ঠীবদ্ধ মানুষ কতগুলি অলিখিত নিয়ম কানুন ও রীতিনীতি সূত্রে একে অন্যের সঙ্গে পরস্পর গাঁতা হয়ে পৃথিবীর এক এক অঞ্চলে একত্রে বসবাস করতে আরম্ভ করলো। এই ভাবেই একদিন সুসভ্য মানুষের সমাজ গড়ে উঠলো। সমাজের উদ্ভবের প্রথম দিন থেকেই মানুষ সমাজ সেবায় প্রয়োজন অনুভব করে। আজ সভ্যতার চরম উৎকর্ষের দিনেও মানব সমাজে এই সেবা ও সহযোগিতার প্রয়োজন এতটুকু কমেনি। সুতরাং সমাজসেবা বলতে সমাজের অন্তর্গত মানুষের সেবা ও কল্যাণকেই বোঝায়।"
sentence1="সমাজসেবার প্রয়োজন সব দেশে থাকলেও ভারতবর্ষের মতো দরিদ্র উন্নয়নশীল বিশাল দেশে এর গুরুত্ব সমাধি। ১০০ কোটি জন অধ্যুষিত এই বিশাল দেশের কোটি কোটি মানুষ নিদারুণ দুঃখ দুর্দশা ও অভাব-অনটনে নিপীড়িত। দেশের অনেক মানুষই আজও নিরক্ষরতার নিবিড় অন্ধকারে নিমজ্জিত। এর ওপর আছে প্রাকৃতিক বিপর্যয় ঝড়-ঝঞ্ঝা খরা, বন্যা, ভূমিকম্প, ও দুর্ভিক্ষ মহামারীর তান্ডব লীলা। বলা বাহুল্য, এদেশে এসব সমস্যার ও দুর্যোগ এর মোকাবিলা করার প্রাথমিক দায়িত্ব দেশের সরকারের বিশেষত আমাদের দেশে যখন জনকল্যাণমুখী গণতান্ত্রিক রাষ্ট্র প্রতিষ্ঠিত। কিন্তু কী অর্থে ,কি সামর্থ্যে এই বিরাট সমস্যার সমাধান করা রাষ্ট্রশক্তির একার পক্ষে কখনোই সম্ভব নয়। সেই দিক দিয়ে এদেশের সমাজ সেবার গুরুত্ব অপরিসীম।"
sentence2="পাঠিকা ঠাকুরানি নিজ গুণে ক্ষমা করবেন। মাতালের গল্প ছাড়া আমার গতি নেই।"
sentence1=sentence1.replace("-"," ")
sentences=sentence1.split("।")
sentences
for sentence in sentences:
    words.extend(nltk.word_tokenize(sentence))

In [ ]:
for i in words:
    if len(i)<2:
        words.remove(i)
for word in words:
    if words.index(word)==0:
        left_context=''
        right_context=words[words.index(word)+1]
    elif words.index(word)==len(words)-1:
        left_context=words[words.index(word)-1]
        right_context=''
    else:
        left_context=words[words.index(word)-1]
        right_context=words[words.index(word)+1]
    prediction_list=[]
    leematizer(word)            

input  সমাজসেবার
['সমাজসেবা']
সমাজসেবার ---> সমাজসেবা
input  প্রয়োজন
['প্রয়োজন']
প্রয়োজন ---> প্রয়োজন
input  সব
['সে', 'সব']


In [ ]:
word='সেবায়'
print(word.)
leematizer(word)


In [ ]:
prediction_list=[]
container=[]
leematizer('আমাকে')
translation_window=translator.translate('সে আমাকে বলেছিল').lower()
print(translation_window)
for translation_testword in ['আমি', 'আম']:
    translation_testword=translator.translate('সে '+translation_testword+' বলেছিল').lower()
    print(translation_testword)
    print(LCsequence(translation_window,translation_testword)/len(translation_testword))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword)))
#    print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/len(translation_testword))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/abs(len(translation_testword)-len(translation_window)))
    print(abs(len(translation_testword)-len(translation_window))/LCsequence(translation_window,translation_testword))

In [ ]:
prediction_list=[]
container=[]
leematizer('গায়ের')

translation_window=translator.translate('তার গায়ের রঙ ').lower()
print(translation_window)
for translation_testword in ['গাওয়া', 'গা']:
    translation_testword=translator.translate('তারে '+translation_testword+' রঙ').lower()
    print(translation_testword)
# print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword)))
    print(LCsequence(translation_window,translation_testword)/len(translation_testword))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/abs(len(translation_testword)-len(translation_window)))
    print(abs(len(translation_testword)-len(translation_window))/LCsequence(translation_window,translation_testword))

In [ ]:
translation_testword=translator.translate('তারে গাওয়া রঙ').lower()
translation_window='the color of his skin'
if translation_testword:
    print(translation_testword)
print(abs(len(translation_testword)-len(translation_window))/LCsequence(translation_window,translation_testword))